Let's take a look at the functional tissue units (FTUs) from each organ.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
from PIL import Image

In [ ]:
df = pd.read_csv('../input/hubmap-organ-segmentation/train.csv')
plt.ylabel('# of images')
df.organ.hist();

In [ ]:
TRAIN_ROOT = '../input/hubmap-organ-segmentation/train_images'
to_tensor = torchvision.transforms.ToTensor()


def get_image(row):
    id = row.id
    img_pil = Image.open(f'{TRAIN_ROOT}/{id}.tiff')
    return to_tensor(img_pil)


def get_mask(row):
    width = row.img_width
    height = row.img_height
    mask = torch.zeros(height * width)
    for p, length in torch.tensor([int(x) for x in row.rle.split(' ')]).reshape(-1, 2):
        mask[p:p+length] = 1.0
    mask = mask.reshape(height, -1).T
    return mask


def get_image_with_mask(row):
    img = get_image(row)
    mask = get_mask(row)
    height, width = mask.shape
    img_with_mask = torch.zeros(3, height, width)
    img_with_mask[0] = mask
    img_with_mask[1:3] = img.mean(0)
    return img_with_mask


to_pil = torchvision.transforms.ToPILImage()
resize = torchvision.transforms.Resize((250, 250))
for organ in df.organ.unique():
    print('\n' + organ)
    display(to_pil(torchvision.utils.make_grid(
        [resize(get_image_with_mask(df[df.organ == organ].iloc[i])) for i in range(16)],
        nrow=4
    )))